In [4]:
import pandas as pd
import re
import os

In [5]:
pd.set_option('display.max_columns', None)

In [7]:
file = "190808_Panama_agosto_19.cohort.combined.hf.vcf"
file2 = "/home/laura/ANALYSIS/SNPTB/190913_Cepas_Ana_Asturias/VCF/190913_Cepas_Ana_Asturias.cohort.combined.hf.vcf"

In [8]:
def import_VCF42_cohort_pandas(vcf_file, sep='\t'):
    """
    Script to read vcf 4.2 cohort/join called vcf handling header lines
    """
    header_lines = 0
    with open(vcf_file) as f:
        first_line = f.readline().strip()
        next_line = f.readline().strip()
        while next_line.startswith("##"):
            header_lines = header_lines + 1
            #print(next_line)
            next_line = f.readline()
    
    if first_line.endswith('VCFv4.2'):
        dataframe = pd.read_csv(vcf_file, sep=sep, skiprows=[header_lines], header=header_lines)
    else:
        print("This vcf file is not v4.2")
        sys.exit(1)
           
    return dataframe

In [16]:
def identify_non_genotyped(vcf_file, nocall_fr=0.5):
    
    df = import_VCF42_cohort_pandas(vcf_file)
    
    non_genotyped_positions = []
    
    sample_list = df.columns[9:].tolist()
    #remove positions which haven't been enotyped in 0.2% or more samples
    for index, data_row in df.iloc[:,9:].iterrows():
        if any(x.startswith("./.") for x in data_row):
            #print(data_row.tolist())
            non_genotyped = [x.startswith("./.") for x in data_row] #True False array
            non_genotyped_count = sum(non_genotyped) #True = 1, False = 0
            if non_genotyped_count / len(non_genotyped) > nocall_fr:
                non_genotyped_positions.append(df.loc[index, 'POS'])
                #print(df.loc[index, 'POS'], is_heterozygous_count, len(is_heterozygous))
        
    return non_genotyped_positions
    

In [17]:
identify_non_genotyped(file)

[39022,
 39030,
 104767,
 150890,
 150896,
 150897,
 335971,
 336155,
 336257,
 336459,
 336473,
 336480,
 336504,
 336772,
 336773,
 336780,
 336781,
 336782,
 427098,
 623270,
 623273,
 623298,
 672212,
 836303,
 836309,
 836314,
 836315,
 837218,
 837220,
 837223,
 837467,
 839515,
 839516,
 839519,
 839520,
 839534,
 840235,
 890718,
 925537,
 925811,
 925813,
 925819,
 1189829,
 1189838,
 1217675,
 1342063,
 1468916,
 1468917,
 1468920,
 1632547,
 1632554,
 1632558,
 1633531,
 1633533,
 1633590,
 1634523,
 1637088,
 1637089,
 1637090,
 1637096,
 1637097,
 1637102,
 1637105,
 1637192,
 1637197,
 1637656,
 1779283,
 1788570,
 1983291,
 1986652,
 1986656,
 2061974,
 2163790,
 2165953,
 2165956,
 2165962,
 2165963,
 2165965,
 2165967,
 2165968,
 2165980,
 2165981,
 2165988,
 2165989,
 2430137,
 2634003,
 2638997,
 2693290,
 2693377,
 2802041,
 2829294,
 2961572,
 2961605,
 2961758,
 2972117,
 2973463,
 3054659,
 3054663,
 3054694,
 3054752,
 3127931,
 3232703,
 3248300,
 3248304,
 324

In [18]:
identify_non_genotyped(file2)

[104276,
 104280,
 104283,
 336773,
 336787,
 338719,
 623270,
 623273,
 836414,
 836416,
 837218,
 837220,
 837223,
 927072,
 927073,
 1096299,
 1189829,
 1189838,
 1342063,
 1468916,
 1468917,
 1468920,
 1633523,
 1633531,
 1633533,
 1780800,
 1781551,
 1991713,
 2197230,
 2197237,
 2197239,
 2262620,
 2423819,
 2424165,
 2693377,
 2784613,
 3803109,
 3846946,
 3846947,
 3878578,
 3884562,
 3934127,
 3935409,
 3935449,
 3936086,
 3936122,
 3948404,
 3948414,
 3948418,
 3948419,
 3948939,
 4253578,
 33706,
 336771,
 336777,
 336781,
 336785,
 1189839,
 1633538,
 1633543,
 1633545,
 1779281,
 1780535,
 1780541,
 1780586,
 2784617,
 2972109,
 2972110,
 3122696,
 3552714,
 3552717,
 3710383,
 3710384,
 3710387,
 3710389,
 3795059,
 3795063,
 3795066,
 3846928,
 3846932,
 3846935,
 3878579,
 3936088,
 3936094,
 3936097,
 3936105,
 3936107,
 3936115,
 3944498,
 3948927,
 3948928]

In [53]:
test = ['0|1:40,0:40:99:0,102,1355', '0/3:33,0:33:99:0,99,1031', '0/0:52,0:52:99:0,120,1800', '0/1:45,0:45:99:0,107,1665', '0/0:57,0:57:99:0,113,1751', '0/0:35,0:35:93:0,93,1395', '0/0:43,0:43:99:0,110,1450', '0/0:50,0:50:99:0,113,1694', '0/0:54,0:54:99:0,117,1800']

In [55]:
if any(bool(re.search(r'0[|\/][1-9]', x)) for x in test):
    print("YES")
    test_l = [bool(re.search(r'0[|\/][1-9]', x)) for x in test]
    print(test_l)
else:
    print("NO")

YES
[True, True, False, True, False, False, False, False, False]


In [52]:
is_test = [bool(re.search(r'0[|\/]1', x)) for x in test] #True False array

In [36]:
is_test

[True, False, True, False, False, False, False, False, False]

In [185]:
df_test = import_VCF42_cohort_pandas(file)

In [186]:
df = repare_nongenotyped(file)

PA117-18C 21819 ./.:1,0:1:.:0,0,0
PA-004 132804 ./.:10,0:10:.:.:.:0,0,0
T-18-008 335642 ./.:1,0:1:.:0,0,0
PA-212 336325 ./.:1,0:1:.:.:.:0,0,0
P120 672330 ./.:3,0:3:.:.:.:0,0,0
P120 672332 ./.:3,0:3:.:.:.:0,0,0
PA197-18C 673564 ./.:1,0:1:.:0,0,0
T-18-008 673564 ./.:5,0:5:.:0,0,0
PA-004 839216 ./.:3,0:3:.:.:.:0,0,0
PA184-18C 839581 ./.:35,0:35:.:.:.:0,0,0
PA202-18C 839581 ./.:33,0:33:.:.:.:0,0,0
P107 839584 ./.:22,0:22:.:.:.:0,0,0
PA184-18C 839584 ./.:33,0:33:.:.:.:0,0,0
P107 839586 ./.:22,0:22:.:.:.:0,0,0
PA184-18C 839586 ./.:33,0:33:.:.:.:0,0,0
PA184-18C 1000794 ./.:59,0:59:.:.:.:0,0,0
T18-040 1093850 ./.:2,0:2:.:0,0,0
PA197-18C 1093954 ./.:1,0:1:.:.:.:0,0,0
T-18-008 1094204 ./.:1,0:1:.:0,0,0
T18-040 1094204 ./.:1,0:1:.:0,0,0
P46 1095644 ./.:13,0:13:.:.:.:0,0,0
P46 1095669 ./.:17,0:17:.:.:.:0,0,0
P46 1095672 ./.:16,0:16:.:.:.:0,0,0
PA117-18C 1095990 ./.:2,0:2:.:.:.:0,0,0
PA149-18C 1095990 ./.:4,0:4:.:.:.:0,0,0
T-18-008 1096398 ./.:1,0:1:.:.:.:0,0,0
T18-003 1189567 ./.:7,0:7:.:.:.:0,0,0

In [67]:
test = ['./.:17,0:17:.:.:.:0,0,0', '0/0:4,0:4:12:.:.:0,12,140', '0/0:10,0:10:30:.:.:0,30,405', '0/0:16,0:16:48:.:.:0,48,570', '0/0:44,0:44:0:.:.:0,0,15', '0/0:3,0:3:9:.:.:0,9,82', '0|1:2,4:6:99:0|1:150890_G_A:162,0,385:150890', './.:26,0:26:.:.:.:0,0,0', '0/0:19,0:19:57:.:.:0,57,663', './.:42,0:42:.:.:.:0,0,0', './.:33,0:33:.:.:.:0,0,0', './.:15,0:15:.:.:.:0,0,0', './.:11,0:11:.:.:.:0,0,0', './.:34,0:34:.:.:.:0,0,0', './.:18,0:18:.:.:.:0,0,0', '0|1:1,13:14:99:0|1:150890_G_A:495,0,127:150890', './.:31,0:31:.:.:.:0,0,0', './.:36,0:36:.:.:.:0,0,0', './.:17,0:17:.:.:.:0,0,0', './.:20,0:20:.:.:.:0,0,0', './.:16,0:16:.:.:.:0,0,0', '0/0:5,0:5:0:.:.:0,0,101', '0/0:9,0:9:27:.:.:0,27,342', './.:16,0:16:.:.:.:0,0,0', './.:19,0:19:.:.:.:0,0,0', '0/0:10,0:10:0:.:.:0,0,316']

In [68]:
test2 = any(x.startswith("./.") for x in test)

In [70]:
test2 = [x.startswith("./.") for x in test]

In [73]:
sum(test2)

15

In [74]:
len(test2)

26

In [18]:
df.columns[9:].tolist()

['1104',
 '1209',
 'P107',
 'P118',
 'P120',
 'P46',
 'P97',
 'PA-004',
 'PA-204',
 'PA-212',
 'PA117-18C',
 'PA119-18C',
 'PA136-18C',
 'PA149-18C',
 'PA155-18C',
 'PA157-18C',
 'PA164-18C',
 'PA176-18C',
 'PA184-18C',
 'PA197-18C',
 'PA202-18C',
 'T-18-008',
 'T18-003',
 'T18-027',
 'T18-040',
 'T18-055']

In [ ]:
d.apply(lambda x: x.str.contains('|'.join(v)))

In [ ]:
d.applymap(lambda x: any([x.startswith(v) for v in vals]))


In [79]:
def list_to_bed(input_list, output_dir, output_file_name, reference="CHROM"):
    """
    Turn a list into a bed file with start and end position having the same value
    """
    output_dir = os.path.abspath(output_dir)
    
    output_bed_file = output_file_name + ".bed"
    
    final_output_path = os.path.join(output_dir, output_bed_file)
    
    with open (final_output_path, 'w+') as f:
        for position in input_list:
            line = ("\t").join([reference, str(position), str(position)]) + "\n"
            f.write(line)
        
    

In [80]:
list_het = identify_heterozygous(file)

In [81]:
list_to_bed(list_het, ".", "is_heterozygous")

In [78]:
%%bash

pwd

/home/laura/DEVELOP/SNPTB/test
